In [ ]:
source extend_bashkernel-2modes.source
set-global-evaluation-mode "my-script"

In [ ]:
set-global-evaluation-mode "my-script"

# Creating a rpm package

To learn how the rpm packages works, we will go through the process of creating a minimal package from scratch.

## Task: Install packages

### Task details

Make sure the required packages are installed. For this sample install the following packages
* git
* rpm-build
* rpmlint

### Example

To install a package we can make use of yum.
```
yum install package_name
```

#### Provided script

In [ ]:
# Task: create-rpm-install-packages
# Evaluation Mode: provided-script

# Install required packages

ssh -qi ../mykeypair root@10.0.2.100 "sudo yum install -y git rpm-build rpmlint"

### Check

In [ ]:
# Task: create-rpm-install-packages
# Evaluation Mode: check

## Task: Setup rpmbuild folders

### Task details

rpmbuild makes use of certain folders, so in order for it to work properly create the following folders 
```
${HOME}/rpmbuild/BUILD
${HOME}/rpmbuild/BUILDROOT
${HOME}/rpmbuild/RPMS
${HOME}/rpmbuild/SOURCES
${HOME}/rpmbuild/SPECS
${HOME}/rpmbuild/SRPMS
```

### Example

to create a folder make use of the inbuild ```mkdir``` command

#### Provided script

In [ ]:
# Task: create-rpm-create-folders
# Evaluation Mode: provided-script

# Create the folders used by rpmbuild

ssh -qi ../mykeypair root@10.0.2.100 'mkdir -p ${HOME}/rpmbuild/{BUILD,BUILDROOT,RPMS,SOURCES,SPECS,SRPMS}'

### Check

In [ ]:
# Task: create-rpm-create-folders
# Evaluation Mode: check

## Task: Create the spec file

### Task details

One of two requirements of to create rpms is the rpmspec file. This file has to be placed in the folder
```
${HOME}/rpmbuild/SPECS
```
It has to be a .spec file and the name should be the same as the package name. For our package create a file with the following contents in the SPECS folder. Because our package is called example, name the file example.spec
```
Name:    example
Version: 0.1.0
Release: 1%{?dist}
Summary: example
License: BSD
Source:  %{name}-%{version}.tar.gz

%description

%prep
%setup -q

%build

%install
rm   -rf $RPM_BUILD_ROOT
mkdir -p $RPM_BUILD_ROOT
rsync -avx ./ $RPM_BUILD_ROOT/

%clean
rm -rf $RPM_BUILD_ROOT

%files
%defattr(-,root,root,-)
/
%doc

%changelog```

### Example

* To move between folders make use of the ```cd``` command.
* To create files use the ```touch``` command.

(Explain how to use here-docs/login and use vi)

#### Provided script

In [ ]:
# Task: create-rpm-create-spec-file
# Evaluatoin Mode: provided-script

# Create the spec file

ssh -qi ../mykeypair root@10.0.2.100 <<'EOS'
cat <<'EOF' > ${HOME}/rpmbuild/SPECS/example.spec
Name:    example
Version: 0.1.0
Release: 1%{?dist}
Summary: example
License: BSD
Source:  %{name}-%{version}.tar.gz

%description

%prep
%setup -q

%build

%install
rm   -rf $RPM_BUILD_ROOT
mkdir -p $RPM_BUILD_ROOT
rsync -avx ./ $RPM_BUILD_ROOT/

%clean
rm -rf $RPM_BUILD_ROOT

%files
%defattr(-,root,root,-)
/
%doc

%changelog
EOF
EOS

### Check

In [ ]:
# Task: create-rpm-create-spec-file
# Evaluation Mode: check

## Task: Check the spec file

### Task details

To verify that all the required fields are included in the spec file we can use the command ```rpmlint```
Use it with the newly created example.spec to confirm that there are no errors.
The result should look like this
```
example.spec: W: no-buildroot-tag
example.spec: W: invalid-url Source0: example-0.1.0.tar.gz
0 packages and 1 specfiles checked; 0 errors, 2 warnings.
```


### Example

```rpmlint package.spec```

#### Provided script

In [4]:
# Task: create-rpm-check-required-fields
# Evaluation Mode: provided-script

# Chech required fields

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS/
rpmlint example.spec
EOF

Pseudo-terminal will not be allocated because stdin is not a terminal.
example.spec: W: no-buildroot-tag
example.spec: W: invalid-url Source0: example-0.2.0.tar.gz
0 packages and 1 specfiles checked; 0 errors, 2 warnings.


### Check

In [ ]:
# Task: create-rpm-check-required-fields
# Evaluation Mode: check

## Task: Create package contents

### Task details

The other requirement for rpms are the archive containing the source code. First the contents of the archive will need to be created.

* Create the folder ```example-0.1.0``` in ```${HOME}/rpmbuild/SOURCES```
* Create the folder ```bin``` inside the newly created ```example-0.1.0```
* Create an empty file named ```example``` inside the new ```bin``` folder

Use the ```find``` command and confirm that the output looks like this:
```
example-0.1.0/bin/example
```

### Example

```find package -type -f```

#### Provided script

In [ ]:
# Task: create-rpm-prepare-contents
# Evaluation Mode: provided-script

# Prepare the contents and create a package

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SOURCES
mkdir example-0.1.0
mkdir example-0.1.0/bin
touch example-0.1.0/bin/example
find example-0.1.0 -type f
EOF

### Check

In [ ]:
# Task: create-rpm-prepare-contents
# Evaluation Mode: check

## Task: Create the archive

### Task details

Next create the the archive using the ```tar``` command. The rule when creating an archive for rpms is to include the package name and the version.

### Example

the package should be named like this
```
tar czvf package-0.0.0.tar.gz package-0.0.0
```

#### Provided script

In [ ]:
# Task: create-rpm-create-tar
# Evaluation Mode: provided-script

# Prepare the contents and create a package

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SOURCES
tar zcvf example-0.1.0.tar.gz example-0.1.0
EOF

### Check

In [ ]:
# Task: create-rpm-create-tar
# Evaluation Mode: provided-script

## Task: Build the rpm

### Task details

Use the rpmbuild too to create the rpm file, the output should look similar to this:
```
Executing(%prep): /bin/sh -e /var/tmp/rpm-tmp.thSbon
+ umask 022
+ cd /home/vagrant/rpmbuild/BUILD
+ cd /home/vagrant/rpmbuild/BUILD
+ rm -rf example-0.1.0
+ /bin/tar -xf -
+ /usr/bin/gzip -dc /home/vagrant/rpmbuild/SOURCES/example-0.1.0.tar.gz
+ STATUS=0
+ '[' 0 -ne 0 ']'
+ cd example-0.1.0
+ /bin/chmod -Rf a+rX,u+w,g-w,o-w .
+ exit 0
Executing(%build): /bin/sh -e /var/tmp/rpm-tmp.eYFskt
+ umask 022
+ cd /home/vagrant/rpmbuild/BUILD
+ cd example-0.1.0
+ exit 0
Executing(%install): /bin/sh -e /var/tmp/rpm-tmp.cwEehz
+ umask 022
+ cd /home/vagrant/rpmbuild/BUILD
+ cd example-0.1.0
+ rm -rf /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
+ mkdir -p /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
+ rsync -avx ./ /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64/
sending incremental file list
./
bin/
bin/example

sent 128 bytes  received 38 bytes  332.00 bytes/sec
total size is 0  speedup is 0.00
+ /usr/lib/rpm/brp-compress
+ /usr/lib/rpm/brp-strip
+ /usr/lib/rpm/brp-strip-static-archive
+ /usr/lib/rpm/brp-strip-comment-note
Processing files: example-0.1.0-1.el6.x86_64
Requires(rpmlib): rpmlib(CompressedFileNames) <= 3.0.4-1 rpmlib(PayloadFilesHavePrefix) <= 4.0-1
Checking for unpackaged file(s): /usr/lib/rpm/check-files /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
warning: Could not canonicalize hostname: vagrant-centos6
Wrote: /home/vagrant/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
Executing(%clean): /bin/sh -e /var/tmp/rpm-tmp.6TMilN
+ umask 022
+ cd /home/vagrant/rpmbuild/BUILD
+ cd example-0.1.0
+ rm -rf /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
+ exit 0```

### Example

```
rpmbuild -bb package.spec
```

#### Provided script

In [ ]:
# Task: create-rpm-build-rpm
# Evaluation Mode: provided-script

# Build the rpm file

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS
rpmbuild -bb example.spec
EOF

### Check

In [ ]:
# Task: create-rpm-build-rpm
# Evaluation Mode: check

## Task: Install package from rpm

### Task details

In the previous task we got the following ouput
```
Wrote: /home/vagrant/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
```
which tells us where our newly created rpm file was placed, use ```rpm``` and try to install the file.\
the output should look similar to this:
```
Preparing...                ########################################### [100%]
   1:example                ########################################### [100%]
```

with the commands ```rpm -qi example``` and ```rpm -ql example``` we can get the package information and a list of what the package contained.

### Example

```sudo -ihv /path/to/rpm/file```

#### Provided script

In [ ]:
# Task: create-rpm-install-from-rpm
# Evaluatoin Mode: provided-script

# Install and confirm the package contents/information

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo rpm -ivh /root/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
rpm -qi example
rpm -ql example
ls -l /bin/example
EOF

### Check

In [ ]:
# Task: create-rpm-install-from-rpm
# Evaluation Mode: check

## Task

### Task details

Now use the ```rpm``` command to remove the example package.
By doing so ```rpm -qi example``` should output that the package is no longer installed

### Example

```sudo rpm -e package```

#### Provided script

In [ ]:
# Task: create-rpm-remove-package
# Evaluation Mode: provided-script

# Remove the package

ssh -qi ../mykeypair root@10.0.2.100 'sudo rpm -e example'

### Check

In [ ]:
# Task: create-rpm-remove-packages
# Evaluation Mode: check

In [8]:
python --version
ssh -i ../mykeypair root@10.0.2.100 python --version

Python 3.5.1 :: Anaconda 2.4.1 (64-bit)
Python 2.6.6
